In [4]:
from langdetect import detect
import sys
import sqlite3

In [5]:
dbname = 'mod_trip_advisor.db'

with sqlite3.connect(dbname) as conn:
    cur = conn.cursor()
    qry = """SELECT key from reviews WHERE country IN ('Laos')
            """
    cur.execute(qry)
    key_list = cur.fetchall()


In [6]:
#update all the languages
#cursor.execute(qry, vars) - vars needs to be a list
with sqlite3.connect(dbname) as conn:
    cur = conn.cursor()
    for ix, key in enumerate(key_list):
        if ix % 10 == 0:
            sys.stdout.flush()
            sys.stdout.write("\r idx: % d" % ix)
        qry = """SELECT review_text FROM reviews WHERE key = '{}' """.format(key[0])
        cur.execute(qry)
        review = cur.fetchall()[0][0]
        
        try:
            lang = detect(review)
        except langdetect.lang_detect_exception.LangDetectException:
            continue
        qry = """UPDATE reviews SET lang = '{}'
                WHERE key = '{}'""".format(lang, key[0])
        cur.execute(qry)
        if ix%10 == 0:
            conn.commit()

        

 idx:  26100

In [99]:
# Save the longform name of the languages
with open('lang_dict.txt') as f:
    lines = f.read()

In [113]:
newlines = lines.split("\n")
lang_dict = {}
for line in newlines:
    groups = line.split()
    shortlang = groups[0].replace('hl=','')
    longlang = " ".join(groups[1:])
    lang_dict[shortlang] = longlang

with open('langdict.json','w') as f:
    json.dump(lang_dict, f)

In [114]:
with open('langdict.json') as f:
    lines = f.read()

data = json.loads(lines)